# Filtrado

In [ ]:
import re
import pandas as pd
import nltk
import time
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from string import punctuation
from multiprocessing import Pool
import spacy
class Procesamiento:
    def __init__(self,nombreArchivo,nombreHashtag):    
      df=pd.read_csv(nombreArchivo,lineterminator='\n')
      df['tweet']=df['tweet'].apply(lambda x: str(x).lower())
      print(f"tamano del df {df.shape[0]}")
      print(f"filtrando el archivo \"{nombreHashtag}\"")
    # format='%Y%m%d',
      #df['newTime']=df['date'].astype(str) + 'T' + df['time'].astype(str)
      #df['date'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
      #---df['date'] = pd.to_datetime(df['date']+' '+df['time'], errors='coerce')
      self.tweets=df[["date","time","tweet","language","likes_count"]].copy()

      #retweets_count,likes_count,replies_count
      self.tweets=self.tweets.sort_values(by=['likes_count'],ascending=False)
      self.tweets=self.tweets[self.tweets['likes_count']>=0]
      """------------Filtrar tweets en ingles------------"""
      
      self.tweets=self.tweets[self.tweets['language']=="en"]
      print(f"tamano de solo palabras en ingles {self.tweets.shape[0]}")
      self.tweets.drop('language', axis=1, inplace=True)
      """------------Filtro caractestiticas Redes Sociales------------"""
      #emojis
      self.tweets['tweet']=self.tweets['tweet'].apply(lambda x: self.convert_emojis(str(x)))

      self.corpus=self.tweets['tweet'].to_list()
      #numeros
      #self.corpus=[re.sub('[0-9]+', '', word) for word in self.corpus]
      # enlaces paginas
      self.corpus=[re.sub('http[s]?://\S+', '', document) for document in self.corpus]

      """------------Normalizar y  tokenizar------------"""
      #self.corpus=[ str(document).lower() for document in self.corpus]
      
      #tokenizar
      self.corpus=[[word for word in str(document).split()] for document in self.corpus]
      """------------Punctuacion------------"""
      my_punct = ['!', '"', '$', '%', '&', '\'', '(', ')', '*', '+', ',', '.','/', ':', ';', '<',
                  '=', '>', '?', '[', '\\', ']', '^', '`', '{', '|', '}', '~', '»', '«',
                  '“', '”','’','…','‘','\"','–']
      my_punct+=['#','@','-','_']
      punct_pattern = re.compile("[" + re.escape("".join(my_punct)) + "]")
      self.corpus=[[re.sub(punct_pattern, "", word) for word in document] for document in self.corpus]
      """------------stopwords------------"""
      #stopwords
      palabrasEvitar=["ur","ya","of","russia's","russian","dm","rt", "via", "amp",
                      "http", "https", "m", "re", "co","-","--",
                      f"#{str(nombreHashtag).lower()}",str(nombreHashtag[1:]).lower()]
      palabrasAdicionales=["#wwdc22","qatar"]
      stop_words=[]
      with open("/content/drive/MyDrive/Proyectos 2/CODE/stopwords.txt", "r") as sw:
        for line in sw:
          stop_words.append(line.strip())


      stop_words+=palabrasEvitar
      stop_words+=palabrasAdicionales

      self.corpus=[[word for word in document if word not in stop_words] for document in self.corpus]
      """------------Lemmatizer------------"""
      wnl = WordNetLemmatizer()
      self.corpus=[[wnl.lemmatize(word) for word in document] for document in self.corpus]

      self.corpus=[' '.join(document) for document in self.corpus]
      #---------------- GUARDAR RESULTADO
      self.export()
      nombreExportar=nombreArchivo[0:-(len(nombreHashtag)+9)]
      print(f"exportando a {nombreExportar}procesado/{nombreHashtag}Proc.csv")
      self.tweets.to_csv(f"{nombreExportar}procesado/{nombreHashtag}Proc.csv", encoding='utf-8',index=False)
    def convert_emojis(self,string):
      emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  
                                u"\U0001F300-\U0001F5FF"  
                                u"\U0001F680-\U0001F6FF"  
                                u"\U0001F1E0-\U0001F1FF"  
                                u"\U00002500-\U00002BEF"  
                                u"\U00002702-\U000027B0"
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                u"\U0001f926-\U0001f937"
                                u"\U00010000-\U0010ffff"
                                u"\u2640-\u2642"
                                u"\u2600-\u2B55"
                                u"\u200d"
                                u"\u23cf"
                                u"\u23e9"
                                u"\u231a"
                                u"\ufe0f"  
                                u"\u3030"
                                "]+", flags=re.UNICODE)
      return emoji_pattern.sub(r'', string)
    def export(self):

      #self.corpus=[[word for word in document]for document in self.corpus]
      
      self.tweets['tweetFiltrado']=self.corpus
      naRows=list(self.tweets[self.tweets['tweetFiltrado'] == ''].index)
      print(f"Tamanio filas : {self.tweets.shape[0]}")
      if naRows:
        print(f"Se encontraron {len(naRows)} vacios ")
        self.tweets.drop(naRows, axis=0, inplace=True)
        print(f"Tamanio filas : {self.tweets.shape[0]}")


      

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Filtrado

In [ ]:
dir="/content/drive/MyDrive/Proyectos 2/CODE/data/"
hashtags=["#russia","#texas","#WWDC","qatar"]

print("*********\t FILTRADO \t*********")
for file in hashtags:
  start_time = time.time()
  result=Procesamiento(dir+file+".csv",file)
  print(f"Tiempo: {time.time() - start_time} segundos")
  #result.tweets.head(10)
  print("-"*12)

*********	 FILTRADO 	*********


<ipython-input-2-c4c0387436d0>:7: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  result=Procesamiento(dir+file+".csv",file)


tamano del df 81938
filtrando el archivo "#russia"
tamano de solo palabras en ingles 57732
Tamanio filas : 57732
Se encontraron 32 vacios 
Tamanio filas : 57700
exportando a /content/drive/MyDrive/Proyectos 2/CODE/procesado/#russiaProc.csv
Tiempo: 27.909127473831177 segundos
------------
tamano del df 86971
filtrando el archivo "#texas"
tamano de solo palabras en ingles 59092
Tamanio filas : 59092
Se encontraron 134 vacios 
Tamanio filas : 58958
exportando a /content/drive/MyDrive/Proyectos 2/CODE/procesado/#texasProc.csv
Tiempo: 23.53804898262024 segundos
------------
tamano del df 76404
filtrando el archivo "#WWDC"
tamano de solo palabras en ingles 38942
Tamanio filas : 38942
exportando a /content/drive/MyDrive/Proyectos 2/CODE/procesado/#WWDCProc.csv
Tiempo: 8.750253438949585 segundos
------------
tamano del df 497061
filtrando el archivo "qatar"
tamano de solo palabras en ingles 178266
Tamanio filas : 178266
Se encontraron 402 vacios 
Tamanio filas : 177864
exportando a /content/dr

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Proyectos 2/CODE/procesado/qatarProc.csv",lineterminator='\n')
df.shape

(177864, 5)

In [ ]:
df.head()

,date,time,tweet,likes_count,tweetFiltrado
0,2022-12-01,08:19:07,great to be in qatar for the #worldcup2022. t...,298268,great worldcup2022 great hospitality perfect o...
1,2022-12-02,12:16:05,qatar really cooked up one of the best world c...,163719,cooked best world cup
2,2022-12-02,12:05:16,nah man qatar cooked us the best world cup gro...,92607,nah man cooked best world cup group stage
3,2022-12-01,16:01:16,@qatari @thfckei because focusing on football ...,73393,qatari thfckei focusing football
4,2022-11-30,12:03:13,im organising flying my bugatti from dubai to ...,72414,organising flying bugatti dubai turning world ...
